<center>
<img src="../ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
<center>Автор материала: программист-исследователь Mail.Ru Group Юрий Кашницкий

# <center> Домашнее задание № 8
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План
    1. Введение
    2. Описание данных
    3. Предобработка данных
    4. Обучение и проверка моделей
    5. Заключение

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 data/stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l data/stackoverflow.10kk.tsv

10000000 data/stackoverflow.10kk.tsv
CPU times: user 2.67 s, sys: 566 ms, total: 3.24 s
Wall time: 1min 42s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [23]:
import os
from tqdm import tqdm, tqdm_notebook
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. 10 Гб у меня обработались примерно за 2 минуты.

In [20]:
!python preprocess.py data/stackoverflow.10kk.tsv data/stackoverflow.vw

100%|████████████████████████████| 10000000/10000000 [06:36<00:00, 25195.37it/s]


In [21]:
%%time
!wc -l data/stackoverflow.vw

4389054 data/stackoverflow.vw
CPU times: user 1.31 s, sys: 289 ms, total: 1.6 s
Wall time: 47.4 s


In [ ]:
!gzip data/stackoverflow.10kk.tsv

Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [30]:
%%time
!head -n 1463018 data/stackoverflow.vw > data/stackoverflow_train.vw
!head -n 2926036 data/stackoverflow.vw | tail -n 1463018 > data/stackoverflow_valid.vw
!tail -n 1463018 data/stackoverflow.vw > data/stackoverflow_test.vw

!cat data/stackoverflow_valid.vw | cut -f1 -d '|' > data/stackoverflow_valid_labels.txt
!cat data/stackoverflow_test.vw | cut -f1 -d '|' > data/stackoverflow_test_labels.txt

CPU times: user 6.16 s, sys: 1.25 s, total: 7.41 s
Wall time: 4min 29s


In [31]:
%%time
!wc -l data/stackoverflow_train.vw
!wc -l data/stackoverflow_valid.vw
!wc -l data/stackoverflow_test.vw
!wc -l data/stackoverflow_valid_labels.txt
!wc -l data/stackoverflow_valid_test.txt

1463018 data/stackoverflow_train.vw
1463018 data/stackoverflow_valid.vw
1463018 data/stackoverflow_test.vw
1463018 data/stackoverflow_valid_labels.txt
1463018 data/stackoverflow_valid_test.txt
CPU times: user 1.79 s, sys: 365 ms, total: 2.15 s
Wall time: 1min 2s


### 4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: `loss_function=hinge`, `bit_precision`=28 и `seed`=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [44]:
passes = [1, 3, 5]
ngram = [1, 2, 3]

params = [(p, n) for p in passes for n in ngram]

with open('data/stackoverflow_valid_labels.txt', 'r') as file:
    valid_labels = [int(label.strip()) for label in file]

best_score = 0
best_params = None

for (p, n) in tqdm_notebook(params):
    print('learn: passes={}; ngram={}'.format(p, n))
    !vw --oaa 10 -d data/stackoverflow_train.vw --loss_function=hinge --bit_precision 28 --random_seed 17 \
        -f data/model/valid_model.vw --cache --passes $p --ngram $n 
    print('test on valid...')
    !vw -i data/model/valid_model.vw -t -d data/stackoverflow_valid.vw \
        -p data/model/valid_predictions.txt
    with open('data/model/valid_predictions.txt', 'r') as file:
        test_labels = [int(label.strip()) for label in file]
    acc = accuracy_score(valid_labels, test_labels)
    print('accuracy={}'.format(acc))
    if acc > best_score:
        best_params = (p, n)
        best_score = acc

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

learn: passes=1; ngram=1

learn: passes=1; ngram=1
Generating 1-grams for all namespaces.
final_regressor = data/model/valid_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/stackoverflow_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.750000 0.750000           16           16.0        7        7      209
0.781250 0.812500           32           32.0        7        2      174
0.765625 0.750000           64           64.0        3        3      204
0.648438 0.531250          128          128.0        1        

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/home/voudy/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/voudy/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/voudy/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



0.108549 0.096210       524288       524288.0        1        1      818
0.099817 0.091085      1048576      1048576.0        1        1      571

finished run
number of examples = 1463018
weighted example sum = 1463018.000000
weighted label sum = 0.000000
average loss = 0.095764
total feature number = 291954690
test on valid...
Generating 1-grams for all namespaces.
only testing
predictions = data/model/valid_predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/stackoverflow_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        2        2      178
0.000000 0.000000            2            2.0        7        7       74
0.000000 0.000000            4            4.0        5        5      259
0.000000 0.000000            8            8.0        7    

0.160690 0.129211        65536        65536.0        5        5      430
0.136086 0.111481       131072       131072.0        7        2      760
0.115993 0.095901       262144       262144.0        7        7      298
0.102705 0.089417       524288       524288.0        1        1     2449
0.090337 0.077969      1048576      1048576.0        1        1     1708

finished run
number of examples = 1463018
weighted example sum = 1463018.000000
weighted label sum = 0.000000
average loss = 0.085885
total feature number = 868548985
test on valid...
Generating 3-grams for all namespaces.
only testing
predictions = data/model/valid_predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/stackoverflow_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        2    

0.457031 0.380859         1024         1024.0        1        1      132
0.383301 0.309570         2048         2048.0        7        7       71
0.310059 0.236816         4096         4096.0        2        2      319
0.262085 0.214111         8192         8192.0        5        5       24
0.223450 0.184814        16384        16384.0        3        3      581
0.185547 0.147644        32768        32768.0        3        3       28
0.158493 0.131439        65536        65536.0        4        4      184
0.137115 0.115738       131072       131072.0        2        2       95
0.121109 0.105103       262144       262144.0        5        5      232
0.108404 0.095699       524288       524288.0        6        6      142
0.100007 0.091610      1048576      1048576.0        1        1      422
0.092582 0.092582      2097152      2097152.0        5        5      696 h
0.088155 0.083728      4194304      4194304.0        1        1      216 h

finished run
number of examples per pass = 131

0.687500 0.656250           64           64.0        7        7      304
0.640625 0.593750          128          128.0        5        1      823
0.625000 0.609375          256          256.0        1        1      301
0.544922 0.464844          512          512.0        2        1      199
0.466797 0.388672         1024         1024.0        1        1      391
0.402344 0.337891         2048         2048.0        7        7      208
0.328125 0.253906         4096         4096.0        2        2      952
0.279419 0.230713         8192         8192.0        5        2       67
0.232727 0.186035        16384        16384.0        3        3     1738
0.191132 0.149536        32768        32768.0        3        3       79
0.159424 0.127716        65536        65536.0        4        4      547
0.135880 0.112335       131072       131072.0        2        2      280
0.117374 0.098869       262144       262144.0        5        5      691
0.102125 0.086876       524288       524288.0      

In [48]:
best_score, best_params

(0.932356949811964, (1, 2))

<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [60]:
!vw --oaa 10 -d data/stackoverflow_train.vw --loss_function hinge --bit_precision 28 --random_seed 17 \
        -f data/model/best_model.vw --cache --passes 1 --ngram 2 --save_resume

Generating 2-grams for all namespaces.
final_regressor = data/model/best_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using cache_file = data/stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.750000 0.718750           64           64.0        3        3      406
0.648438 0.546875          128          128.0        1        7       56
0.6171

In [57]:
!vw -i data/model/best_model.vw -t -d data/stackoverflow_test.vw \
        -p data/model/test_predictions.txt

Generating 2-grams for all namespaces.
only testing
predictions = data/model/test_predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        9        9      370
0.000000 0.000000            2            2.0        1        1       62
0.250000 0.500000            4            4.0        5        5      158
0.250000 0.250000            8            8.0        2        2      116
0.125000 0.000000           16           16.0        6        6       74
0.093750 0.062500           32           32.0        1        1      324
0.078125 0.062500           64           64.0        4        4       52
0.078125 0.078125          128          128.0        7        7      250
0.070312 0.062500         

In [58]:
with open('data/stackoverflow_test_labels.txt', 'r') as file:
    valid_labels = [int(label.strip()) for label in file]
with open('data/model/test_predictions.txt', 'r') as file:
        test_labels = [int(label.strip()) for label in file]
test_score = accuracy_score(valid_labels, test_labels)
test_score

0.9325346646452743

In [66]:
(best_score - test_score) * 100

-0.017771483331030513

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [76]:
!cat data/stackoverflow_train.vw data/stackoverflow_valid.vw | \
    vw  -f data/model/combine_model.vw \
    --oaa 10 --loss_function hinge --bit_precision 28 --random_seed 17 \
    --cache --passes 1 --ngram 2

Generating 2-grams for all namespaces.
final_regressor = data/model/combine_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = .cache
Reading datafile = 
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.750000 0.718750           64           64.0        3        3      406
0.648438 0.546875          128          128.0        1        7       56
0.617188 0.585938          256          256.0      

In [77]:
!vw -i data/model/combine_model.vw -t -d data/stackoverflow_test.vw \
        -p data/model/combine_predictions.txt

Generating 2-grams for all namespaces.
only testing
predictions = data/model/combine_predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        9        9      370
0.000000 0.000000            2            2.0        1        1       62
0.000000 0.000000            4            4.0        5        5      158
0.000000 0.000000            8            8.0        2        2      116
0.000000 0.000000           16           16.0        6        6       74
0.031250 0.062500           32           32.0        1        1      324
0.046875 0.062500           64           64.0        4        4       52
0.046875 0.046875          128          128.0        7        7      250
0.058594 0.070312      

In [78]:
with open('data/stackoverflow_test_labels.txt', 'r') as file:
    valid_labels = [int(label.strip()) for label in file]
with open('data/model/combine_predictions.txt', 'r') as file:
    test_labels = [int(label.strip()) for label in file]
combine_score = accuracy_score(valid_labels, test_labels)
combine_score

0.9364471250524601

In [79]:
(combine_score - test_score) * 100

0.3912460407185736

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

### 5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)